In [2]:
!pip install chrono

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os

# Setting up output directory
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# List to store output files
output_files = []

# Loading and cleaning data
def load_and_clean_data(file_path, sample_size=10000):
    try:
        # Load CSV without parsing dates initially
        df = pd.read_csv(file_path)
        # Sample data to reduce computation time
        if len(df) > sample_size:
            df = df.sample(n=sample_size, random_state=42)
            print(f"\nSampled {sample_size} rows from dataset.")

        # Inspect 'time' column
        print("\nSample 'time' column values (first 5):")
        print(df['time'].head())

        # Parse 'time' column to datetime
        try:
            df['time'] = pd.to_datetime(df['time'], errors='coerce')
        except Exception as e:
            print(f"Error parsing 'time' column: {e}")
            print("Problematic 'time' values:")
            invalid_times = df[df['time'].isna()]['time']
            print(invalid_times.head(10))
            return None

        # Print dataset info and description before cleaning
        print("\nDataset Info (Before Cleaning):")
        df.info()
        print("\nDataset Description (Before Cleaning):")
        print(df.describe())

        # Drop rows with NaT in 'time' or NaN in any column
        df = df.dropna()
        df = df[df['time'].notnull()]

        # Convert numeric columns
        numeric_columns = ['Cyclone_Inlet_Gas_Temp', 'Cyclone_Material_Temp', 'Cyclone_Outlet_Gas_draft',
                           'Cyclone_cone_draft', 'Cyclone_Gas_Outlet_Temp', 'Cyclone_Inlet_Draft']
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df.dropna(subset=numeric_columns)

        # Print dataset info and description after cleaning
        print("\nDataset Info (After Cleaning):")
        df.info()
        print("\nDataset Description (After Cleaning):")
        print(df.describe())

        return df

    except FileNotFoundError:
        print(f"Error: File {file_path} not found. Please check the path.")
        return None
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# Feature Engineering
def engineer_features(df, columns):
    df_eng = df.copy()
    df_eng['Temp_Diff_Inlet_Outlet'] = df['Cyclone_Inlet_Gas_Temp'] - df['Cyclone_Gas_Outlet_Temp']
    for col in columns:
        df_eng[f'{col}_rolling_mean'] = df[col].rolling(window=12).mean()
        df_eng[f'{col}_rolling_std'] = df[col].rolling(window=12).std()
    df_eng = df_eng.dropna()

    print("\nDataset Info (After Feature Engineering):")
    df_eng.info()
    print("\nDataset Description (After Feature Engineering):")
    print(df_eng.describe())

    return df_eng

# Feature Selection
def select_features(df, columns):
    corr_matrix = df[columns].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > 0.8)]

    iso = IsolationForest(random_state=42)
    iso.fit(df[columns])
    importances = np.abs(iso.decision_function(df[columns])).mean(axis=0)
    importance_df = pd.DataFrame({'Feature': columns, 'Importance': importances})
    importance_file = os.path.join(output_dir, 'feature_importance.csv')
    importance_df.to_csv(importance_file, index=False)
    output_files.append(importance_file)

    print("\nFeature Importance:")
    print(importance_df)

    selected_features = [col for col in columns if col not in to_drop]
    print("\nSelected Features:", selected_features)

    return selected_features

# Model Checking
def check_models(df, features):
    scaler = StandardScaler()
    X = scaler.fit_transform(df[features])

    models = {
        'IsolationForest': IsolationForest(n_estimators=100, contamination=0.01, random_state=42),
        'OneClassSVM': OneClassSVM(nu=0.01, gamma='scale'),
        'LOF': LocalOutlierFactor(n_neighbors=20, contamination=0.01, novelty=True),
        'DBSCAN': DBSCAN(eps=0.5, min_samples=5)
    }

    def build_autoencoder(input_dim):
        model = Sequential([
            Dense(16, activation='relu', input_dim=input_dim),
            Dense(8, activation='relu'),
            Dense(16, activation='relu'),
            Dense(input_dim, activation='linear')
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    results = {}
    anomaly_labels_dict = {}
    print("\nModel Evaluation Metrics:")
    for model_name, model in models.items():
        silhouette = None  # Initialize silhouette to None
        if model_name == 'LOF':
            model.fit(X)
            anomaly_labels = model.predict(X)
            scores = -model.negative_outlier_factor_
            try:
                silhouette = silhouette_score(X, anomaly_labels)
            except ValueError:
                silhouette = -1  # Handle single cluster or invalid cases
        elif model_name == 'DBSCAN':
            labels = model.fit_predict(X)
            anomaly_labels = np.where(labels == -1, 1, 0)
            scores = np.abs(model.fit(X).components_.sum(axis=1)) if hasattr(model, 'components_') else np.zeros(len(X))
            if len(np.unique(labels)) > 1:
                silhouette = silhouette_score(X, labels)
            else:
                silhouette = -1
        else:
            model.fit(X)
            anomaly_labels = np.where(model.predict(X) == -1, 1, 0)
            scores = -model.decision_function(X)

        df[f'{model_name}_anomaly'] = anomaly_labels
        anomaly_labels_dict[model_name] = anomaly_labels
        anomaly_count = np.sum(anomaly_labels)
        anomaly_proportion = anomaly_count / len(X)
        score_mean = np.mean(scores)
        score_std = np.std(scores)

        print(f"\n{model_name}:")
        print(f"  Number of Anomalies Detected: {anomaly_count}")
        print(f"  Anomaly Proportion: {anomaly_proportion:.4f}")
        print(f"  Mean Anomaly Score: {score_mean:.4f}")
        print(f"  Std Anomaly Score: {score_std:.4f}")
        if model_name in ['LOF', 'DBSCAN'] and silhouette is not None:
            print(f"  Silhouette Score: {silhouette:.4f}" if silhouette != -1 else "  Silhouette Score: Not applicable (single cluster)")

        results[model_name] = {'scores': scores, 'silhouette': silhouette if model_name in ['LOF', 'DBSCAN'] else None}

    autoencoder = build_autoencoder(len(features))
    autoencoder.fit(X, X, epochs=10, batch_size=32, verbose=0)
    preds = autoencoder.predict(X, verbose=0)
    mse = np.mean(np.power(X - preds, 2), axis=1)
    threshold = np.percentile(mse, 95)
    anomaly_labels = np.where(mse > threshold, 1, 0)
    df['Autoencoder_anomaly'] = anomaly_labels
    anomaly_labels_dict['Autoencoder'] = anomaly_labels

    print(f"\nAutoencoder:")
    print(f"  Number of Anomalies Detected: {np.sum(anomaly_labels)}")
    print(f"  Anomaly Proportion: {np.sum(anomaly_labels) / len(X):.4f}")
    print(f"  Mean Reconstruction Error: {np.mean(mse):.4f}")
    print(f"  Std Reconstruction Error: {np.std(mse):.4f}")
    print(f"  Reconstruction Error Threshold (95th percentile): {threshold:.4f}")

    results['Autoencoder'] = {'scores': mse, 'threshold': threshold}

    # Compute cross-model agreement
    model_names = list(models.keys()) + ['Autoencoder']
    overlap_matrix = np.zeros((len(model_names), len(model_names)))
    for i, m1 in enumerate(model_names):
        for j, m2 in enumerate(model_names):
            if i <= j:
                overlap = np.mean(anomaly_labels_dict[m1] & anomaly_labels_dict[m2])
                overlap_matrix[i, j] = overlap
                overlap_matrix[j, i] = overlap
    print("\nCross-Model Anomaly Agreement (Proportion of Overlapping Anomalies):")
    overlap_df = pd.DataFrame(overlap_matrix, index=model_names, columns=model_names)
    print(overlap_df)

    return results

# Aggregating anomaly periods
def aggregate_anomaly_periods(df, model_name, time_threshold='30min'):
    anomaly_periods = []
    anomalies = df[df[f'{model_name}_anomaly'] == 1][['time']].copy().reset_index(drop=True)
    if anomalies.empty:
        print(f"\nNo anomalies detected by {model_name}.")
        return anomaly_periods

    anomalies['time_diff'] = anomalies['time'].diff().dt.total_seconds().fillna(0)
    current_period_start = None
    for i, row in anomalies.iterrows():
        if current_period_start is None:
            current_period_start = row['time']
        elif row['time_diff'] > pd.Timedelta(time_threshold).total_seconds():
            anomaly_periods.append((current_period_start, anomalies.loc[i-1, 'time']))
            current_period_start = row['time']
    if current_period_start is not None:
        anomaly_periods.append((current_period_start, anomalies['time'].iloc[-1]))

    print(f"\nAnomaly Periods for {model_name}:")
    for start, end in anomaly_periods:
        print(f"  Start: {start}, End: {end}")

    return anomaly_periods

# Visualizing anomalies
def plot_anomalies(df, model_names, columns, output_dir):
    for model_name in model_names:
        anomaly_file = os.path.join(output_dir, f'anomalies_{model_name}.png')
        plt.figure(figsize=(10, 6))
        for col in columns:
            plt.plot(df['time'], df[col], label=col, alpha=0.5)
            anomalies = df[df[f'{model_name}_anomaly'] == 1]
            plt.scatter(anomalies['time'], anomalies[col], label=f'{col} anomalies', s=50)
        plt.xlabel('Time')
        plt.ylabel('Values')
        plt.title(f'Anomalies Detected by {model_name}')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(anomaly_file)
        plt.close()
        output_files.append(anomaly_file)
        print(f"\nGenerated Image: {anomaly_file}")
        print(f"  Description: Time-series plot of variables with anomalies marked for {model_name}")

# Plotting model evaluation graphs
def plot_model_evaluation(df, results, model_names, output_dir):
    # Anomaly score distributions
    plt.figure(figsize=(12, 8))
    for model_name in model_names:
        scores = results[model_name]['scores']
        sns.histplot(scores, label=model_name, alpha=0.5, bins=50)
    plt.xlabel('Anomaly Score (or Reconstruction Error)')
    plt.ylabel('Frequency')
    plt.title('Anomaly Score Distributions')
    plt.legend()
    score_dist_file = os.path.join(output_dir, 'anomaly_score_distribution.png')
    plt.savefig(score_dist_file)
    plt.close()
    output_files.append(score_dist_file)
    print(f"\nGenerated Image: {score_dist_file}")
    print("  Description: Histogram of anomaly scores (or reconstruction errors) for all models")

    # Silhouette scores
    silhouette_scores = {m: results[m]['silhouette'] for m in ['LOF', 'DBSCAN'] if results[m]['silhouette'] is not None and results[m]['silhouette'] != -1}
    if silhouette_scores:
        plt.figure(figsize=(8, 6))
        plt.bar(silhouette_scores.keys(), silhouette_scores.values(), color=['#1f77b4', '#ff7f0e'])
        plt.xlabel('Model')
        plt.ylabel('Silhouette Score')
        plt.title('Silhouette Scores for LOF and DBSCAN')
        silhouette_file = os.path.join(output_dir, 'silhouette_scores.png')
        plt.savefig(silhouette_file)
        plt.close()
        output_files.append(silhouette_file)
        print(f"\nGenerated Image: {silhouette_file}")
        print("  Description: Bar plot of silhouette scores for LOF and DBSCAN")

    # Cross-model agreement
    model_names_list = list(model_names)
    overlap_matrix = np.zeros((len(model_names), len(model_names)))
    for i, m1 in enumerate(model_names):
        for j, m2 in enumerate(model_names):
            if i <= j:
                overlap = np.mean(df[f'{m1}_anomaly'] & df[f'{m2}_anomaly'])
                overlap_matrix[i, j] = overlap
                overlap_matrix[j, i] = overlap

    plt.figure(figsize=(10, 8))
    sns.heatmap(overlap_matrix, annot=True, xticklabels=model_names_list, yticklabels=model_names_list, cmap='Blues')
    plt.title('Cross-Model Anomaly Agreement')
    overlap_file = os.path.join(output_dir, 'cross_model_agreement.png')
    plt.savefig(overlap_file)
    plt.close()
    output_files.append(overlap_file)
    print(f"\nGenerated Image: {overlap_file}")
    print("  Description: Heatmap of proportion of overlapping anomalies across models")

# Main execution
def main():
    file_path = '/content/data (5) (1) (1) (1) (1)(internship-data-1).csv'
    columns = ['Cyclone_Inlet_Gas_Temp', 'Cyclone_Material_Temp', 'Cyclone_Outlet_Gas_draft',
               'Cyclone_cone_draft', 'Cyclone_Gas_Outlet_Temp', 'Cyclone_Inlet_Draft']

    # Loading and cleaning data
    df = load_and_clean_data(file_path)
    if df is None:
        return

    # Feature Engineering
    df = engineer_features(df, columns)
    all_features = columns + ['Temp_Diff_Inlet_Outlet'] + [f'{col}_rolling_mean' for col in columns] + [f'{col}_rolling_std' for col in columns]

    # Feature Selection
    selected_features = select_features(df, all_features)

    # Standardizing features
    scaler = StandardScaler()
    df[all_features] = scaler.fit_transform(df[all_features])

    # Model Checking
    results = check_models(df, selected_features)

    # Saving anomaly periods and plotting
    model_names = ['IsolationForest', 'OneClassSVM', 'LOF', 'DBSCAN', 'Autoencoder']
    for model_name in model_names:
        anomaly_periods = aggregate_anomaly_periods(df, model_name)
        anomaly_df = pd.DataFrame(anomaly_periods, columns=['Start_Time', 'End_Time'])
        anomaly_file = os.path.join(output_dir, f'anomaly_periods_{model_name}.csv')
        anomaly_df.to_csv(anomaly_file, index=False)
        output_files.append(anomaly_file)

    plot_anomalies(df, model_names, columns, output_dir)
    plot_model_evaluation(df, results, model_names, output_dir)

    # Printing output files
    print("\nGenerated Output Files:")
    for file in output_files:
        print(f"- {file}")

if __name__ == "__main__":
    main()


Sampled 10000 rows from dataset.

Sample 'time' column values (first 5):
130073    3/29/2018 15:20
359408     6/4/2020 22:25
75214      9/19/2017 3:50
248621     5/16/2019 6:15
218887      2/2/2019 0:25
Name: time, dtype: object

Dataset Info (Before Cleaning):
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 130073 to 301102
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   time                      9970 non-null   datetime64[ns]
 1   Cyclone_Inlet_Gas_Temp    9970 non-null   object        
 2   Cyclone_Material_Temp     9970 non-null   object        
 3   Cyclone_Outlet_Gas_draft  9970 non-null   object        
 4   Cyclone_cone_draft        9970 non-null   object        
 5   Cyclone_Gas_Outlet_Temp   9970 non-null   object        
 6   Cyclone_Inlet_Draft       9970 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 625.0+ KB

Dataset 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Streaming output truncated to the last 5000 lines.
  Start: 2018-07-05 01:40:00, End: 2018-07-05 01:40:00
  Start: 2018-07-29 06:00:00, End: 2018-07-29 06:00:00
  Start: 2020-01-22 11:30:00, End: 2017-12-08 13:50:00
  Start: 2018-12-23 21:15:00, End: 2017-12-16 14:10:00
  Start: 2019-07-12 14:45:00, End: 2019-07-12 14:45:00
  Start: 2019-08-08 20:35:00, End: 2018-08-31 23:25:00
  Start: 2019-08-29 14:00:00, End: 2017-05-10 16:05:00
  Start: 2020-01-14 20:10:00, End: 2017-11-26 05:05:00
  Start: 2019-07-13 19:55:00, End: 2017-11-03 05:00:00
  Start: 2020-04-04 21:30:00, End: 2018-01-23 17:45:00
  Start: 2019-06-19 04:15:00, End: 2019-06-19 04:15:00
  Start: 2020-08-04 19:00:00, End: 2018-08-12 06:45:00
  Start: 2018-10-24 14:15:00, End: 2018-10-24 14:15:00
  Start: 2020-01-21 19:15:00, End: 2018-05-25 23:05:00
  Start: 2019-08-25 01:15:00, End: 2019-08-25 01:15:00
  Start: 2020-02-03 05:55:00, End: 2018-02-04 20:10:00
  Start: 2018-02-20 02:50:00, End: 2018-02-20 02:50:00
  Start: 2018-